# U-Net Model for Semantic Segmentation (with DenseNet121 Example)

Paper: https://arxiv.org/abs/2009.02805

This document describes the architecture and data flow of a U-Net model for semantic image segmentation, using DenseNet121 as the encoder. The model is implemented in PyTorch.

## 1. Introduction

Semantic segmentation is the task of assigning a class label to each pixel in an image. U-Net architectures are particularly well-suited for this because they capture both high-level contextual information and fine-grained spatial details. This implementation uses a pre-trained DenseNet121 as the encoder, leveraging transfer learning for improved performance.

## 2. Model Architecture

## DenseNet-U-Net Model Flowchart

This flowchart visualizes the architecture and data flow of a DenseNet-U-Net model for image segmentation.

    
    A[Input: (batch_size, 3, 256, 256)] --> B{Encoder (DenseNet121)}

    B --> C[Layer 0: conv0, norm0, relu0]
    C --> D[Layer 1: pool0, denseblock1]
    D --> E[Layer 2: transition1, denseblock2]
    E --> F[Layer 3: transition2, denseblock3]
    F --> G[Layer 4: transition3, denseblock4, norm5, relu5]

    G --> H{Bottleneck (ConvBottleneck)}
    H --> I[Concatenate with Upsampled Decoder Output]

    I --> J{Decoder Stage 1}
    J --> K[Upsample, Conv2d, ReLU]
    K --> L[Concatenate with Encoder Layer 3 Output]

    L --> M{Decoder Stage 2}
    M --> N[Upsample, Conv2d, ReLU]
    N --> O[Concatenate with Encoder Layer 2 Output]

    O --> P{Decoder Stage 3}
    P --> Q[Upsample, Conv2d, ReLU]
    Q --> R[Concatenate with Encoder Layer 1 Output]

    R --> S{Decoder Stage 4 (Last Upsample)}
    S --> T[Upsample, Conv2d, ReLU]

    T --> U[Final Layer: 1x1 Conv]
    U --> V[Output: (batch_size, num_classes, 256, 256)]

In [38]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/kaggle/img-classif")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -r requirements.txt

In [39]:
os.chdir("./unet_pipeline")

In [41]:
!pip install -U albumentations

  Using cached albumentations-2.0.3-py3-none-any.whl.metadata (38 kB)
  Using cached albucore-0.0.23-py3-none-any.whl.metadata (5.3 kB)
Using cached albumentations-2.0.3-py3-none-any.whl (284 kB)
Using cached albucore-0.0.23-py3-none-any.whl (14 kB)
  Attempting uninstall: albucore
    Found existing installation: albucore 0.0.19
    Uninstalling albucore-0.0.19:
      Successfully uninstalled albucore-0.0.19
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.4.20
    Uninstalling albumentations-1.4.20:
      Successfully uninstalled albumentations-1.4.20


In [49]:
!ls ../input/dataset1024

mask  test  train


In [50]:
!python Train.py experiments/albunet_valid/train_config_part0.yaml

{'PIPELINE_NAME': 'albunet_1024', 'DATA_DIRECTORY': '../input/dataset1024', 'LOGGER_DIR': 'log', 'SEED': 42, 'DEVICE': 'cuda', 'DEVICE_LIST': [0], 'WORKERS': 12, 'MODEL': {'PRETRAINED': {'PIPELINE_PATH': 'experiments/albunet_valid', 'CHECKPOINTS_FOLDER': 'checkpoints', 'PIPELINE_NAME': 'albunet_1024'}, 'FREEZE': False, 'PY': 'models.ternausnets', 'CLASS': 'AlbuNet', 'ARGS': {'pretrained': False}}, 'FOLD': {'NUMBER': 5, 'USEFOLDS': [0, 1, 2, 3, 4], 'FILE': 'folds/train_folds_5.csv'}, 'USE_SAMPLER': True, 'NON_EMPTY_MASK_PROBA': 0.8, 'TRAIN_TRANSFORMS': 'transforms/train_transforms_complex_1024_old.json', 'VALID_TRANSFORMS': 'transforms/valid_transforms_1024_old.json', 'CRITERION': {'PY': 'Losses', 'CLASS': 'ComboLoss', 'ARGS': {'weights': {'bce': 3, 'dice': 1, 'focal': 4}}}, 'OPTIMIZER': {'CLASS': 'Adam', 'ARGS': {'lr': 0.0001, 'weight_decay': 5e-06}}, 'SCHEDULER': {'CLASS': 'ReduceLROnPlateau', 'ARGS': {'mode': 'max', 'factor': 0.1, 'patience': 2, 'threshold': 1e-07, 'min_lr': 1e-07}},